In [1]:
!pip install --user tf-agents[reverb]

# Use this tag get the matching examples and colabs.
!git clone https://github.com/tensorflow/agents.git
!cd agents
!git checkout v0.12.0

# !pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
# !pip install tf-agents[reverb]
!pip install pyglet

  Using cached tf_agents-0.12.1-py3-none-any.whl (1.3 MB)
  Using cached tensorflow_probability-0.16.0-py2.py3-none-any.whl (6.3 MB)
  Using cached gin_config-0.5.0-py3-none-any.whl (61 kB)
  Using cached gym-0.23.1-py3-none-any.whl
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
     |████████████████████████████████| 18.3 MB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 497.9 MB 61 kB/s /s eta 0:00:01 394.9 MB 108.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 76.0 MB/s eta 0:00:01
  Using cached portpicker-1.5.0-py3-none-any.whl (14 kB)
     |████████████████████████████████| 143 kB 103.0 MB/s eta 0:00:01
  Using cached gym_notices-0.0.6-py3-none-any.whl (2.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.26.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached google_pasta-0.2.0-py3-none-a

In [45]:
import numpy as  np

res = np.array([[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
                [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
                [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
                [1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
                [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
                [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1],
                [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0],
                [1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1],
                [0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
                [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0]])


def veiw_shot_list(res,x,y):
    x, y = np.where(res[x-1:y+2,x-1:y+2] == 0)
    path = np.array([(i,j) for i, j in zip(x, y)]) - (1,1)
    steps = {
            0: (0, 0),
            2: (1, 0),   
            1: (-1, 0),  
            4: (0, 1),   
            3: (0, -1),  
        }
    x_ = [[k for k in steps if (steps[k] == i).all()] for i in path]
    return [i for j in x_ for i in j]

veiw_shot_list(res,5,5)

[1, 0, 2]

In [47]:
res[6,6]

1

In [3]:
!pip install reverb

  Using cached reverb-2.0.1.zip (3.4 kB)
  Created wheel for reverb: filename=reverb-2.0.1-py3-none-any.whl size=3689 sha256=d14eb193022d33307e1e2cf6c18991549c608c6a7c9c42f43caab9925bb3cd22
  Stored in directory: /root/.cache/pip/wheels/32/f5/10/f2e2651a09c36a5d000f4cc763058488a14918292d38c0de1e
Successfully built reverb


In [4]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

SyntaxError: invalid syntax (reverb.py, line 64)

In [ ]:
# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

In [ ]:
import gym
from pogema.wrappers.multi_time_limit import MultiTimeLimit
from pogema.animation import AnimationMonitor
from IPython.display import SVG, display
import numpy as np
import pogema
from pogema import GridConfig

# Define random configuration
grid_config = GridConfig(num_agents=3, # количество агентов на карте
                         size=4,      # размеры карты
                         density=0.4,  # плотность препятствий
                         seed=1,       # сид генерации задания 
                         max_episode_steps=256,  # максимальная длина эпизода
                         obs_radius=5, # радиус обзора
                        )

env = gym.make("Pogema-v0", grid_config=grid_config)
env = AnimationMonitor(env)

# обновляем окружение
obs = env.reset()

done = [False, ...]

while not all(done):
    # Используем случайную стратегию
    obs, reward, done, info = env.step([np.random.randint(4) for _ in range(len(obs))])
# сохраняем анимацию и рисуем ее
env.save_animation("render.svg", egocentric_idx=1)
display(SVG('render.svg'))

In [ ]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)